# Sweep Setup

In [ ]:
#@title Wandb mode
import os
mode = 'online' #@param {type:"string"}
os.environ['WANDB_MODE'] = mode

if 'WANDB_API_KEY' not in os.environ:
    if os.environ['WANDB_MODE'] == 'online':
        from getpass import getpass
        secret = getpass('Enter WandB API Key: ')
        os.environ['WANDB_API_KEY'] = secret
    else:
        print("WandB Offline!")

In [ ]:
#@title Setup
!rm -rf /content/Whats-this-rock/
!git clone https://github.com/udaylunawat/Whats-this-rock.git
# !git clone -b hydra https://github.com/udaylunawat/Whats-this-rock.git
%cd /content/Whats-this-rock/

!nvidia-smi
!sh src/scripts/setup.sh

In [ ]:
!git pull

In [ ]:
%%writefile configs/sweep.yaml
method: bayes
metric:
  goal: minimize
  name: val_loss
parameters:
  notes:
    value: ""
  seed:
    values: [1]
  lr:
    values: [1e-2, 1e-3, 1e-4, 1e-5]
  lr_decay_steps:
    values: [10, 20, 50, 100, 300, 500, 1000, 3000]
  lr_schedule:
    values:
      - cosine_decay_restarts
  epochs:
    value: 150
  augmentation:
    values: [kerascv, True]
  class_weights:
    value: True
  optimizer:
    values: [adam]
  loss:
    values: [mean_squared_error, categorical_crossentropy]
  metrics:
    value: ["accuracy"]
  batch_size:
    value: 64
  num_classes:
    value: 7
  train_split:
    values:
      - 0.75
  data_path:
    value: data/4_tfds_dataset/
  wandb.use:
    value: True
  wandb.mode:
    value: online
  wandb.project:
    value: Whats-this-rockv17
  dataset_id:
    values:
      - [1,2,3,4]
  image_size:
    value: 224
  image_channels:
    value: 3
  sampling:
    values: [None]
  backbone:
    values: [resnet]
  use_pretrained_weights:
    values: [True]
  trainable:
    values: [False]
  last_layers:
    values: [30]
  custom_callback:
    values: [False]
  preprocess:
    value: True
  dropout_rate:
    values: [0.3]
  monitor:
    values: ["val_loss"]
  earlystopping.use:
    values: [False]
  earlystopping.patience:
    values: [10]
  reduce_lr.use:
    values: [False]
  reduce_lr.factor:
    values: [.9]
  reduce_lr.patience:
    values: [1]
  reduce_lr.min_lr:
    values: [1e-5, 1e-6]
  save_model:
    value: False

program: src/models/train.py
command:
  - ${env}
  - python
  - ${program}
  - ${args_no_hyphens}


In [ ]:
# !wandb sweep \
#     --project Whats-this-rockv17 \
#     --entity udaylunawat \
#     configs/sweep.yaml

In [ ]:
!wandb agent udaylunawat/Whats-this-rockv17/xrjlfpy0